In [1]:
with open('deu-eng.txt', 'r', encoding='utf-8') as f:
  lines = f.read().split('\n')

In [2]:
lines[155]

"It's me!\tIch bin's."

In [3]:
len(lines)

169814

In [4]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

In [5]:
num_samples = 10000
num_samples

10000

In [6]:
for line in lines[: min(num_samples, len(lines) - 1)]:
  input_text, target_text = line.split('\t')
  target_text = '\t' + target_text + '\n'
  input_texts.append(input_text)
  target_texts.append(target_text)
  for char in input_text:
    if char not in input_characters:
      input_characters.add(char)
  for char in target_text:
    if char not in target_characters:
      target_characters.add(char)

In [7]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 70
Number of unique output tokens: 87
Max sequence length for inputs: 16
Max sequence length for outputs: 53


In [8]:
input_token_index = dict(
  [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
  [(char, i) for i, char in enumerate(target_characters)])

In [9]:
import numpy as np

encoder_input_data = np.zeros(
  (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
  dtype='float32')
decoder_input_data = np.zeros(
  (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
  dtype='float32')
decoder_target_data = np.zeros(
  (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
  dtype='float32')

In [10]:
encoder_input_data.shape

(10000, 16, 70)

In [11]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
  for t, char in enumerate(input_text):
    encoder_input_data[i, t, input_token_index[char]] = 1.
  for t, char in enumerate(target_text):
    # decoder_target_data is ahead of decoder_input_data by one timestep
    decoder_input_data[i, t, target_token_index[char]] = 1.
    if t > 0:
      # decoder_target_data will be ahead by one timestep
      # and will not include the start character.
      decoder_target_data[i, t - 1, target_token_index[char]] = 1.

In [12]:
encoder_input_data[155].shape

(16, 70)

In [13]:
import keras, tensorflow
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

Using TensorFlow backend.


In [14]:
batch_size = 64  # batch size for training
epochs = 100  # number of epochs to train for
latent_dim = 256  # latent dimensionality of the encoding space/

In [15]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

In [16]:
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [17]:
model = Model(inputs=[encoder_inputs, decoder_inputs], 
              outputs=decoder_outputs)

In [18]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 70)     0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 87)     0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 256), (None, 334848      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 256),  352256      input_2[0][0]                    
                                                                 lstm_1[0][1]               

In [19]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
model.save('seq2seq_eng-ger.h5')

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 6s 772us/step - loss: 1.0253 - val_loss: 1.0631
Epoch 2/100
8000/8000 [==============================] - 5s 674us/step - loss: 0.8035 - val_loss: 0.8607
Epoch 3/100
8000/8000 [==============================] - 5s 687us/step - loss: 0.6754 - val_loss: 0.7612
Epoch 4/100
8000/8000 [==============================] - 6s 703us/step - loss: 0.6108 - val_loss: 0.7182
Epoch 5/100
8000/8000 [==============================] - 6s 713us/step - loss: 0.5634 - val_loss: 0.6787
Epoch 6/100
8000/8000 [==============================] - 6s 714us/step - loss: 0.5297 - val_loss: 0.6485
Epoch 7/100
8000/8000 [==============================] - 6s 722us/step - loss: 0.5028 - val_loss: 0.6271
Epoch 8/100
8000/8000 [==============================] - 6s 715us/step - loss: 0.4802 - val_loss: 0.6105
Epoch 9/100
8000/8000 [==============================] - 6s 740us/step - loss: 0.4599 - val_loss: 0.5916
Epoch 1

8000/8000 [==============================] - 7s 815us/step - loss: 0.1136 - val_loss: 0.6818
Epoch 62/100
8000/8000 [==============================] - 6s 794us/step - loss: 0.1113 - val_loss: 0.69282s - loss: 0.1 - ETA: 1s  - ETA: 0s - loss: 0 - ETA: 0s - loss: 0
Epoch 63/100
8000/8000 [==============================] - 7s 818us/step - loss: 0.1091 - val_loss: 0.6960 0s - 
Epoch 64/100
8000/8000 [==============================] - 6s 793us/step - loss: 0.1073 - val_loss: 0.6975 ETA: 5s - loss: 0.098 - ETA: 5s - loss: 0. - ETA: 4s - loss: 0.1 - ETA: - E - ETA: 1s 
Epoch 65/100
8000/8000 [==============================] - 6s 797us/step - loss: 0.1051 - val_loss: 0.7044
Epoch 66/100
8000/8000 [==============================] - 7s 823us/step - loss: 0.1028 - val_loss: 0.7090
Epoch 67/100
8000/8000 [==============================] - 6s 787us/step - loss: 0.1006 - val_loss: 0.7135: 4s - loss: 0.095 - ETA: 4s - loss: 0.0 - ETA: 4s - - ETA: 0s - lo
Epoch 68/100
8000/8000 [======================

OSError: Unable to create file (unable to open file: name = '/results/seq2seq_eng-ger.h5', errno = 2, error message = 'No such file or directory', flags = 13, o_flags = 302)

In [20]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(
  decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
  [decoder_inputs] + decoder_states_inputs,
  [decoder_outputs] + decoder_states)

In [21]:
# reverse-lookup token index to turn sequences back to characters
reverse_input_char_index = dict(
  (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
  (i, char) for char, i in target_token_index.items())

In [22]:
def decode_sequence(input_seq):
  # encode the input sequence to get the internal state vectors.
  states_value = encoder_model.predict(input_seq)
  
  # generate empty target sequence of length 1 with only the start character
  target_seq = np.zeros((1, 1, num_decoder_tokens))
  target_seq[0, 0, target_token_index['\t']] = 1.
  
  # output sequence loop
  stop_condition = False
  decoded_sentence = ''
  while not stop_condition:
    output_tokens, h, c = decoder_model.predict(
      [target_seq] + states_value)
    
    # sample a token and add the corresponding character to the 
    # decoded sequence
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_char = reverse_target_char_index[sampled_token_index]
    decoded_sentence += sampled_char
    
    # check for the exit condition: either hitting max length
    # or predicting the 'stop' character
    if (sampled_char == '\n' or 
        len(decoded_sentence) > max_decoder_seq_length):
      stop_condition = True
      
    # update the target sequence (length 1).
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, sampled_token_index] = 1.
    
    # update states
    states_value = [h, c]
    
  return decoded_sentence

In [23]:
for seq_index in range(10):
  input_seq = encoder_input_data[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)
  print('-')
  print('Input sentence:', input_texts[seq_index])
  print('Decoded sentence:', decoded_sentence)

-
Input sentence: Hi.
Decoded sentence: Hallo!

-
Input sentence: Hi.
Decoded sentence: Hallo!

-
Input sentence: Run!
Decoded sentence: Lauf!

-
Input sentence: Fire!
Decoded sentence: Feuer!

-
Input sentence: Help!
Decoded sentence: Zu Hülf!

-
Input sentence: Help!
Decoded sentence: Zu Hülf!

-
Input sentence: Stop!
Decoded sentence: Stopp!

-
Input sentence: Wait!
Decoded sentence: Warte!

-
Input sentence: Go on.
Decoded sentence: Mach weiter.

-
Input sentence: Hello!
Decoded sentence: Hallo!



In [24]:
model.save('seq2seq_eng-ger.h5')